In [31]:
import sys
from PIL import Image
from tensorflow import keras
sys.modules['Image'] = Image

In [32]:
from PIL import Image
print(Image.__file__)

import Image
print(Image.__file__)
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model

/home/fs/smartbrowser/lib/python3.5/site-packages/PIL/Image.py
/home/fs/smartbrowser/lib/python3.5/site-packages/PIL/Image.py


In [33]:
import tensorflow as tf
from tensorflow.keras import layers, losses
from keras import backend as K
class Denoise(Model):
  def __init__(self):
    super(Denoise, self).__init__()
    self.encoder = tf.keras.Sequential([
        layers.Input(shape=(256, 256, 3)),
        layers.Conv2D(32, (3,3), activation='relu', padding='same', strides=2),
        layers.Conv2D(16, (3,3), activation='relu', padding='same', strides=2),
        layers.Conv2D(8, (3,3), activation='relu', padding='same', strides=2),
        layers.Conv2D(4, (3,3), activation='relu', padding='same', strides=2),
        layers.Conv2D(2, (3,3), activation='relu', padding='same', strides=2),
    ])

    self.decoder = tf.keras.Sequential([
        layers.Conv2DTranspose(2, kernel_size=3, strides=2, activation='relu', padding='same'),
        layers.Conv2DTranspose(4, kernel_size=3, strides=2, activation='relu', padding='same'),
        layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
        layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
        layers.Conv2DTranspose(32, kernel_size=3, strides=2, activation='relu', padding='same'),
        layers.Conv2D(1, kernel_size= 3, activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Denoise()
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [34]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1.0/255.0)
# load and iterate training dataset
train_it = datagen.flow_from_directory('provided_dataset_train/', class_mode='input', color_mode='rgb', batch_size=32)
valid_it = datagen.flow_from_directory('provided_dataset_validate/', class_mode='input', color_mode='rgb', batch_size=32)
batchX, batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 4560 images belonging to 1 classes.
Found 1000 images belonging to 1 classes.
Batch shape=(32, 256, 256, 3), min=0.000, max=1.000


In [35]:
from keras.callbacks import TensorBoard
autoencoder.encoder.summary()
#autoencoder.decoder.summary()

checkpoint_filepath = "train_ckpt/cp.ckpt"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_acc',
    save_freq="epoch",
    mode='max',
    save_best_only=True)
autoencoder.fit_generator(
                    generator=train_it,
                    validation_data=valid_it,
                    epochs=30,
                    callbacks=[TensorBoard(log_dir='logs/autoencoder')])

autoencoder.save('saved_models_autoencoder/basic_attempt1_deep')

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 64, 64, 16)        4624      
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 32, 32, 8)         1160      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 16, 16, 4)         292       
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 8, 8, 2)           74        
Total params: 7,046
Trainable params: 7,046
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
143/143 [==============================] - 547s 4s/step - loss: 0.1072 - val_loss: 0.0792

In [1]:
import cv2
import numpy as np

img_orig = cv2.imread('provided_dataset_validate/validate/7964.jpg')

img_res = cv2.resize(img_orig,(256,256))

img_model = np.reshape(img_res,[1,256,256,3])
img_model = np.asarray(img_model,dtype='float32')
img_model/=255.0
print(img_model.min(),img_model.max(),img_model.mean())
print(img_model.shape)
#cv2.imshow("loaded image",img)
#cv2.waitKey(0)

0.007843138 1.0 0.83016557
(1, 256, 256, 3)


In [2]:
from tensorflow import keras
loaded = keras.models.load_model("saved_models_autoencoder/basic_attempt1_deep")
out_image = loaded.predict(img_model)
print(out_image.min(),out_image.max(),out_image.mean())
print(out_image.shape)


0.25555444 1.0 0.83652186
(1, 256, 256, 1)


In [4]:
cv2.imshow("predicted_image",out_image[0])
cv2.waitKey(3000)

-1

In [4]:
#from keras.models import Model
# layer_name = 'conv2d_40'
# intermediate_layer_model = Model(inputs=loaded.input,
#                                  outputs=loaded.get_layer(layer_name).output)
# intermediate_output = intermediate_layer_model.predict(img_model)
# print(intermediate_output.min(),intermediate_output.max(),intermediate_output.mean())
# print(intermediate_output.shape)

AttributeError: Layer denoise_6 is not connected, no input to return.

In [9]:
encoded_img = loaded.encoder(img_model).numpy()
print(encoded_img.min(), encoded_img.max(),encoded_img.mean())
print(encoded_img.shape)

0.7090152 27.484896 11.015536
(1, 8, 8, 2)


In [11]:
import os
import tensorflow as tf
from keras.preprocessing import image
img_width = 256
img_height = 256

folder_path = 'provided_dataset_validate/validate/'
images = []
for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)
print(np.array(images).shape)
images = np.vstack(images)
print(images.shape)

(1000, 1, 256, 256, 3)
(1000, 256, 256, 3)


In [20]:
import pandas as pd
import tensorflow as tf
csv_file = 'Lenskart_products_recruitment_challenge.csv'

df = pd.read_csv(csv_file,
                 usecols=['product_id',
                          'parent_category',
                          'frame_shape'],
                 dtype={"product_id": np.int32,
                        "parent_category":"category",
                        "frame_shape":"category"})
print(df.head())
print(df.dtypes)
df['parent_category'] = df.parent_category.cat.codes
df['frame_shape'] = df.frame_shape.cat.codes
print(df.head())
print(df.dtypes)

   product_id parent_category frame_shape
0        7641        eyeframe   Rectangle
1        7643        eyeframe   Rectangle
2        8254        eyeframe   Rectangle
3        8255        eyeframe   Rectangle
4        8256        eyeframe   Rectangle
product_id            int32
parent_category    category
frame_shape        category
dtype: object
   product_id  parent_category  frame_shape
0        7641                1            2
1        7643                1            2
2        8254                1            2
3        8255                1            2
4        8256                1            2
product_id         int32
parent_category     int8
frame_shape         int8
dtype: object


In [22]:
df.to_csv('lenskart_products_categorical.csv')